In [29]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime
import time
from django.forms.models import model_to_dict
import pandas as pd


def scrapper_2_instancia(progressivo=True):

    def calculo_cnj(seq, ano, j, tr, vara):
        vara = str(vara)
        while len(vara) < 4:
            vara = vara + '0'
        n = int(f'{seq}{ano}{j}{tr}{vara}00')
        d = str(98 - n%97)
        if len(d) == 1:
            d = '0' + d
        seq = str(seq)
        while len(seq) < 7:
            seq = '0' + seq
        return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


    driver = Chrome()

    driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
    driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
    driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
    driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
    driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

    vazio = 0
    pacote = 100

    if SegundaInstancia.objects.all().exists():
        if progressivo:
            q = SegundaInstancia.objects.all().order_by('-seq').only('seq')[:pacote]
            q = [i.seq for i in q]
            SEQUENCIAS = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
            ano = SegundaInstancia.objects.all().order_by('-ano').first().ano


    else:
        seq = 2262000

    for seq in SEQUENCIAS:

        params = {

        'conversationId': '',
        'paginaConsulta': '1',
        'localPesquisa.cdLocal': '-1',
        'cbPesquisa': 'NUMPROC',
        'tipoNuProcesso': 'UNIFICADO',
        'dePesquisaNuUnificado': calculo_cnj(seq, ano, 8, 26, 0),
        'dePesquisa': '',
        'uuidCaptcha': '',
        }

        r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)

        res = {'seq': seq,
               'ano': ano,
               'info_url' : r.url}

        sp = bs(r.content, 'html5lib')

        if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
            vazio +=1

        else:
            vazio = 0
            if sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
                res['segredo_de_justica'] = True
            else:
                tabela_dados_gerais = sp.findAll(attrs={'class':'secaoFormBody'})[1].find('tbody')
                for entrada in tabela_dados_gerais.findAll('tr'):
                    try:
                        k = entrada.find('label').get_text()
                        v = entrada.find('span').get_text()
                        res[k] = v
                    except:
                        pass

                tabela_1_instancia = sp.find(text=re.compile('Números de 1ª Instância')).findAllNext('table')[1].find('tbody')
                if tabela_1_instancia:
                    coleta = []
                    for entrada in tabela_1_instancia.findAll('tr'):
                        coleta.append({k: v.get_text().strip() for k,v in zip(['n_primeira_instancia', 'foro', 'vara', 'juiz', 'obs'], entrada.findAll('td')) })    
                    if len(coleta) > 0:
                        res['numeros_1_instancia'] = coleta

                tabela_partes = sp.find(text=re.compile('Partes do Processo')).findNext('table').find('tbody')

                coleta = []
                for entrada in tabela_partes.findAll('tr'):
                    coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                                   'nome': entrada.findAll('td')[1].next.strip()})
                if len(coleta) > 0:
                    res['partes'] = coleta
                if sp.find(attrs={'id':'tabelaTodasMovimentacoes'}):
                    coleta = []
                    for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
                        linhas = entrada.findAll('td')
                        rp = {'ordem': _}
                        rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

                        if linhas[1].find('a'):
                            rp['link0'] = linhas[1].find('a').get('href')

                        if linhas[2].find('a'):
                            rp['link1'] = linhas[2].find('a').get('href')
                            rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
                        else:
                            rp['movimentacaotag'] = linhas[2].next.strip()

                        rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

                        rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
                        coleta.append(rp)
                    if len(coleta)>0:
                        res['movimentacoes'] = coleta

            res = {k.replace(':','').lower().strip(): v for k,v in res.items()}
            res = {unidecode(k).replace(' ','_').replace('_/_','_'): (v.strip() if isinstance(v, str) else v) for k,v in res.items()}

            if 'valor_da_acao' in res.keys():
                res['valor_da_acao'] = float(res['valor_da_acao'].replace('.','').replace(',','.'))

            SegundaInstancia_m = SegundaInstancia(**{k:v for k,v in res.items() if k not in ['partes','numeros_1_instancia', 'movimentacoes' ]})
            SegundaInstancia_m.save()

            if 'partes' in res.keys():
                partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]

                for p in partes:
                    p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                    posicao = Posicao(segunda_instancia=SegundaInstancia_m, parte=p[0][0], posicao=p[1])
                    posicao.save()

            if 'numeros_1_instancia' in res.keys():
                n_1_instancia = [{k:v for k,v in i.items() if v !='-'} for i in res['numeros_1_instancia']]
                Numero1Instancia.objects.bulk_create([Numero1Instancia(**i, segunda_instancia=SegundaInstancia_m) for i in n_1_instancia])

            if 'movimentacoes' in res.keys():
                m = [Movimentacao(**i, segunda_instancia=SegundaInstancia_m) for i in res['movimentacoes']]
                Movimentacao.objects.bulk_create(m)

        if vazio == int(pacote*0.5):
            break

        print(seq, vazio)
        

def scrapper_1_instancia(progressivo=True):

    db.close_old_connections()


    def calculo_cnj(seq, ano, j, tr, vara):
        vara = str(vara)
        while len(vara) < 4:
            vara = '0' + vara
        n = int(f'{seq}{ano}{j}{tr}{vara}00')
        d = str(98 - n%97)
        if len(d) == 1:
            d = '0' + d
        seq = str(seq)
        while len(seq) < 7:
            seq = '0' + seq
        return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


    driver = Chrome()
    driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
    driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
    driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
    driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
    driver.get('https://esaj.tjsp.jus.br/cpopg/open.do')


    for v in [90, 53]:

        pacote = 100

        if PrimeiraInstancia.objects.filter(v=v).exists():
            ano = PrimeiraInstancia.objects.filter(v=v).latest('ano').ano
        else:
            ano = 2019

        if PrimeiraInstancia.objects.filter(ano=ano, v=v, seq__gt=100000).exists():
            q = PrimeiraInstancia.objects.filter(ano=ano, v=v, seq__gt=100000).order_by('-seq').only('seq')[:pacote]
            q = [i.seq for i in q]
            SEQUENCIAS_MAIOR = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
        else:
            SEQUENCIAS_MAIOR = [i for i in range(1000000, 1000000 + pacote*2)]

        if PrimeiraInstancia.objects.filter(ano=ano, v=v, seq__lt=1000000).exists():
            q = PrimeiraInstancia.objects.filter(ano=ano, v=v, seq__lt=1000000).order_by('-seq').only('seq')[:pacote]
            q = [i.seq for i in q]
            SEQUENCIAS_MENOR = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
        else:
            SEQUENCIAS_MENOR = [i for i in range(1, pacote*2)]


        for SEQUENCIAS in (SEQUENCIAS_MENOR, SEQUENCIAS_MAIOR):
            vazio = 0

            for seq in SEQUENCIAS:

                params = {
                'conversationId': '',
                'dadosConsulta.localPesquisa.cdLocal': '-1',
                'cbPesquisa': 'NUMPROC',
                'dadosConsulta.tipoNuProcesso': 'UNIFICADO',
                'dadosConsulta.valorConsultaNuUnificado': calculo_cnj(seq, ano, 8, 26, v),
                'dadosConsulta.valorConsulta': '',
                'uuidCaptcha':'',}

                r = driver.request('GET', 'https://esaj.tjsp.jus.br/cpopg/search.do', params=params)

                res = {'seq': seq,
                       'ano': ano,
                       'v': v,
                       'info_url' : r.url}

                sp = bs(r.content, 'html5lib')

                res['nuProcessoUnificadoFormatado'] = sp.find(attrs={'id':'nuProcessoUnificadoFormatado'}).get('value')


                if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
                    vazio +=1

                else:
                    vazio = 0

                    if "mostrar: 'true'" in str(sp):
                        res['segredo_de_justica'] = True

                    elif sp.find(attrs={'class':'resultadoPaginacao'}):
                        res['problema'] = True

                    else:
                        if sp.find('a', attrs={'title': 'Pasta digital'}) is not None or sp.find('a', attrs={'class': 'linkConsultaSG'}) is not None:
                            res['digital'] = True

                        for item in sp.findAll(
                                attrs={'class': 'labelClass', 'style': 'text-align:right;font-weight:bold;;'}):
                            if item.get('for') == 'tag.dados.rotulo.principal':
                                res['Processo principal'] = item.findNext(attrs={'class': 'processoPrinc'}).get_text()

                            elif item.get_text() == 'Distribuição:':
                                res[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

                                rp = item.findNext('span').findNext()
                                if rp.find('span') is not None:
                                    res['vara'] = rp.find('span').get_text().strip()

                            elif item.get_text() == 'Apensado ao:':
                                res[item.get_text().strip().replace(':', '')] = item.findNext(
                                    attrs={'class': 'processoPaiApenso'}).get_text().strip()

                            elif item.get_text() in ['Incidente:', 'Execução de Sentença:']:
                                res['incidente_ou_execucao'] = item.findNext('span').get_text().strip()
                            else:
                                res[item.get_text().strip().replace(':', '')] = item.findNext('span').get_text().strip()

                            if item.get_text() == 'Recebido em:':
                                res['vara'] = item.findNext('span').findNext('span').get_text()

                        for i in sp.findAll('span', attrs={'class': 'labelClass'}):
                            res[i.get_text().replace(':', '')] = i.next.next.strip()

                        # extinto_suspenso
                        if sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}) is not None:
                            res['extinto_suspenso'] = sp.find('span', attrs={'style': 'color: red; padding-left: 6px;'}).get_text()

                        if 'Distribuição' in res.keys():
                            rp = res['Distribuição'][:10]
                            res['data_inicial'] = datetime.datetime.strptime(rp, '%d/%m/%Y').date()
                        elif 'Recebido em' in res.keys():
                            rp = res['Recebido em'][:10]
                            res['data_inicial'] = datetime.datetime.strptime(rp, '%d/%m/%Y').date()

                        if sp.find('span', attrs={'style': 'color: #FF0000;'}) is not None:
                            res['tramitacao_prioritaria'] = 'Tramitação prioritária'
                        if 'Valor da ação' in res.keys():
                            res['Valor da ação'] = re.sub(r'\s\s+', ' ', res['Valor da ação'])
                            if re.match(r'(.*?)\s', res['Valor da ação']) is not None:
                                res['moeda'] = re.match(r'(.*?)\s', res['Valor da ação']).group(1)
                            if re.search(r'\s(.*)', res['Valor da ação']) is not None:
                                rp = re.search(r'\s(.*)', res['Valor da ação']).group(1).strip()
                                res['valor_acao_float'] = float(rp.replace('.', '').replace(',', '.'))

                            del res['Valor da ação']

                        if sp.find(attrs={'id':'tablePartesPrincipais'}).find('tbody'):
                            coleta = []
                            for entrada in sp.find(attrs={'id':'tablePartesPrincipais'}).find('tbody').findAll('tr'):
                                coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                                               'nome': entrada.findAll('td')[1].next.strip()})
                            if len(coleta) > 0:
                                res['partes'] = coleta

                        coleta = []
                        for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
                            linhas = entrada.findAll('td')
                            rp = {'ordem': _}
                            rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

                            if linhas[1].find('a'):
                                rp['link0'] = linhas[1].find('a').get('href')

                            if linhas[2].find('a'):
                                rp['link1'] = linhas[2].find('a').get('href')
                                rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
                            else:
                                rp['movimentacaotag'] = linhas[2].next.strip()

                            rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

                            rp['movimentacaodescricao'] = linhas[2].find('span').get_text().strip()
                            coleta.append(rp)
                        if len(coleta)>0:
                            res['movimentacoes'] = coleta

                res = {unidecode(k.lower().strip().replace(' ', '_')):v for k,v in res.items()}


                PrimeiraInstancia_m = PrimeiraInstancia(**{k:v for k,v in res.items() if k not in ['partes', 'movimentacoes' ]})
                PrimeiraInstancia_m.save()

                if 'partes' in res.keys():
                    partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]

                    for p in partes:
                        p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                        posicao = Posicao(primeira_instancia=PrimeiraInstancia_m, parte=p[0][0], posicao=p[1])
                        posicao.save()

                if 'movimentacoes' in res.keys():
                    m = [Movimentacao(**i, primeira_instancia=PrimeiraInstancia_m) for i in res['movimentacoes']]
                    Movimentacao.objects.bulk_create(m)

                if vazio == int(pacote*1.1):
                    break

                print(v, seq, vazio)

                
def starter(intervalo):
    while True:
        
        # faz o update das partes relevantes
        df = pd.read_excel('/home/ezequiel/Desktop/controle_partes.xlsx', index_col='id')
        query = Parte.objects.filter(id__in=df[df['pertinente']==1].index)
        query.update(pertinente=True)
        
        # salva a planilha de partes
        db.close_old_connections()
        df = pd.DataFrame(Parte.objects.all().values())
        w  = pd.ExcelWriter('/home/ezequiel/Desktop/controle_partes.xlsx')
        df.to_excel(w, index=False)
        w.save()
        
        
        # faz o scrapper
        scrapper_1_instancia()
        scrapper_2_instancia()
        
        # forma planilhas

        def edita_entrada(entrada, segunda_instancia=False):
            if segunda_instancia:
                numero_1_instancia = ';\n '.join([i.n_primeira_instancia for i in entrada.numero_1_instancia.all()])

            entrada = model_to_dict(entrada)
            entrada['partes_pertinentes'] = ';\n '.join([i.nome for i in entrada['parte'] if i.pertinente])
            entrada['partes_outras'] = ';\n '.join([i.nome for i in entrada['parte'] if not i.pertinente])
            if segunda_instancia:
                entrada['numero_1_instancia'] = numero_1_instancia
            del entrada['parte']

            return entrada

        partes_pertinentes = Parte.objects.filter(pertinente=True)
        primeira_istancia_q = PrimeiraInstancia.objects.filter(posicao__parte__in=partes_pertinentes).prefetch_related('parte')
        primeira_istancia_q = [edita_entrada(x) for x in primeira_istancia_q]
        df = pd.DataFrame(primeira_istancia_q)
        w = pd.ExcelWriter('/home/ezequiel/Desktop/primeira_instancia.xlsx')
        df.to_excel(w, index=False)
        w.save()

        segunda_instancia_q = SegundaInstancia.objects.filter(posicao__parte__in=partes_pertinentes).prefetch_related('parte')
        segunda_instancia_q = SegundaInstancia.objects.filter(posicao__parte__in=partes_pertinentes).prefetch_related('parte')
        segunda_instancia_q = [edita_entrada(x, segunda_instancia=True) for x in segunda_instancia_q]
        df = pd.DataFrame(segunda_instancia_q)
        w = pd.ExcelWriter('/home/ezequiel/Desktop/segunda_instancia.xlsx')
        df.to_excel(w, index=False)
        w.save()
        
        # descança

        for x in range(0, intervalo):    
            time.sleep(x * 60)


starter(10)

    

90 5013 1
90 5014 2
90 5015 3
90 5016 4
90 5017 5
90 5018 6
90 5019 7
90 5020 8
90 5021 9
90 5022 10
90 5023 11
90 5024 12
90 5025 13
90 5026 14
90 5027 15
90 5028 16
90 5029 17
90 5030 18
90 5031 19
90 5032 20
90 5033 21
90 5034 22
90 5035 23
90 5036 24
90 5037 25
90 5038 26
90 5039 27
90 5040 28
90 5041 29
90 5042 30
90 5043 31
90 5044 32
90 5045 33
90 5046 34
90 5047 35
90 5048 36
90 5049 37
90 5050 38
90 5051 39
90 5052 40
90 5053 41
90 5054 42
90 5055 43
90 5056 44
90 5057 45
90 5058 46
90 5059 47
90 5060 48
90 5061 49
90 5062 50
90 5063 51
90 5064 52
90 5065 53
90 5066 54
90 5067 55
90 5068 56
90 5069 57
90 5070 58
90 5071 59
90 5072 60
90 5073 61
90 5074 62
90 5075 63
90 5076 64
90 5077 65
90 5078 66
90 5079 67
90 5080 68
90 5081 69
90 5082 70
90 5083 71
90 5084 72
90 5085 73
90 5086 74
90 5087 75
90 5088 76
90 5089 77
90 5090 78
90 5091 79
90 5092 80
90 5093 81
90 5094 82
90 5095 83
90 5096 84
90 5097 85
90 5098 86
90 5099 87
90 5100 88
90 5101 89
90 5102 90
90 5103 91
90 5104 

OperationalError: (2006, 'MySQL server has gone away')

In [26]:
import pandas as pd
partes_pertinentes = Parte.objects.filter(pertinente=True)

def edita_entrada(entrada):
    if entrada.numero_1_instancia.all().exists():
        numero_1_instancia = ';\n '.join([i.n_primeira_instancia for i in entrada.numero_1_instancia.all()])
    
    entrada = model_to_dict(entrada)
    entrada['partes_pertinentes'] = ';\n '.join([i.nome for i in entrada['parte'] if i.pertinente])
    entrada['partes_outras'] = ';\n '.join([i.nome for i in entrada['parte'] if not i.pertinente])
    entrada['numero_1_instancia'] = numero_1_instancia
    del entrada['parte']

    return entrada

segunda_instancia_q = SegundaInstancia.objects.filter(posicao__parte__in=partes_pertinentes).prefetch_related('parte')
segunda_instancia_q = [edita_entrada(x) for x in segunda_instancia_q]
df = pd.DataFrame(segunda_instancia_q)
df



id      seq   ano                   processo  segredo_de_justica  \
0      230  2262215  2019  2262215-33.2019.8.26.0000               False   
1      649  2262634  2019  2262634-53.2019.8.26.0000               False   
2      728  2262713  2019  2262713-32.2019.8.26.0000               False   
3      858  2262843  2019  2262843-22.2019.8.26.0000               False   
4      868  2262853  2019  2262853-66.2019.8.26.0000               False   
..     ...      ...   ...                        ...                 ...   
159  17673  2267933  2019  2267933-11.2019.8.26.0000               False   
160  17679  2267939  2019  2267939-18.2019.8.26.0000               False   
161  17792  2268052  2019  2268052-69.2019.8.26.0000               False   
162  17879  2268139  2019  2268139-25.2019.8.26.0000               False   
163  18134  2268394  2019  2268394-80.2019.8.26.0000               False   

                                        outros_numeros                 classe  \
0                                            2077/2019  Agravo de Instrumento   
1    0406070-82.1995.8.26.0053, 406070/1995, 376/19...  Agravo de Instrumento   
2                                              73/2019  Agravo de Instrumento   
3                                            5142/2019  Agravo de Instrumento   
4                                            6480/2019  Agravo de Instrumento   
..                                                 ...                    ...   
159                                               None  Agravo de Instrumento   
160                                               None  Agravo de Instrumento   
161          422146/1999, 583.53.1999.422146, 846/1999  Agravo de Instrumento   
162  414257/1994, 16786/2005, 583.53.1994.414257, 9...  Agravo de Instrumento   
163        13399/2010, 583.90.1000.5071135, 13177/2010  Agravo de Instrumento   

                                               assunto  \
0    DIREITO TRIBUTÁRIO - Impostos - ITBI - Imposto...   
1    DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...   
2    DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...   
3    DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...   
4    DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...   
..                                                 ...   
159  DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...   
160     DIREITO CIVIL - Pessoas Jurídicas - Associação   
161  DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...   
162  DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...   
163  DIREITO TRIBUTÁRIO - Impostos - ISS/ Imposto s...   

                                                origem  \
0    Comarca de São Paulo / Foro Fazenda Pública / ...   
1    Comarca de São Paulo / Foro Fazenda Pública / ...   
2    Comarca de São Paulo / Foro Fazenda Pública / ...   
3    Comarca de Marília / Foro de Marília / Vara da...   
4    Comarca de Marília / Foro de Marília / Vara da...   
..                                                 ...   
159  Comarca de São Paulo / Foro Fazenda Pública / ...   
160                                                      
161  Comarca de São Paulo / Foro Fazenda Pública / ...   
162  Comarca de São Paulo / Foro Fazenda Pública / ...   
163  Comarca de São Paulo / Foro das Execuções Fisc...   

                      distribuicao                         relator  \
0    15ª Câmara de Direito Público                  EURÍPEDES FAIM   
1     4ª Câmara de Direito Público           PAULO BARCELLOS GATTI   
2     7ª Câmara de Direito Público  LUIZ SERGIO FERNANDES DE SOUZA   
3     8ª Câmara de Direito Público        JOSÉ MARIA CÂMARA JUNIOR   
4     6ª Câmara de Direito Público              MARIA OLÍVIA ALVES   
..                             ...                             ...   
159                                                                  
160                                                                  
161                                                                  
162                         

In [24]:
entrada = segunda_instancia_q[0]
# entrada = model_to_dict(entrada)
# entrada['partes_pertinentes'] = ';\n '.join([i.nome for i in entrada['parte'] if i.pertinente])
# entrada['partes_outras'] = ';\n '.join([i.nome for i in entrada['parte'] if not i.pertinente])


entrada.n_1_instancia
# n_1_instancia = ';\n '.join([i.n_primeira_instancia for i in entrada.numero_1_instancia.all()])
# n_1_instancia

AttributeError: 'SegundaInstancia' object has no attribute 'n_1_instancia'